In [104]:
# importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
import warnings
warnings.filterwarnings("ignore")
import datetime


In [105]:
# loading data
df=pd.read_csv("marketing_campaign.csv",sep="\t")
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


In [106]:
#summary of the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

In [107]:
# checking for missing values
df.isna().sum()

ID                      0
Year_Birth              0
Education               0
Marital_Status          0
Income                 24
Kidhome                 0
Teenhome                0
Dt_Customer             0
Recency                 0
MntWines                0
MntFruits               0
MntMeatProducts         0
MntFishProducts         0
MntSweetProducts        0
MntGoldProds            0
NumDealsPurchases       0
NumWebPurchases         0
NumCatalogPurchases     0
NumStorePurchases       0
NumWebVisitsMonth       0
AcceptedCmp3            0
AcceptedCmp4            0
AcceptedCmp5            0
AcceptedCmp1            0
AcceptedCmp2            0
Complain                0
Z_CostContact           0
Z_Revenue               0
Response                0
dtype: int64

In [108]:
# dropping missing values
df.dropna(inplace=True)

In [109]:
# checking for duplicates 
df.duplicated().sum()

0

In [110]:
# converting customer date to datetime
df["Dt_Customer"]=pd.to_datetime(df["Dt_Customer"],dayfirst=True)

In [111]:
# creating a column for the period a person has been a customer relative to the newest customer
dates=[]
for i in df["Dt_Customer"].dt.year:
    dates.append(i)
maximum=max(dates)
df["Customer_For"]=maximum-df["Dt_Customer"].dt.year
df["Customer_For"]
    

0       2
1       0
2       1
3       0
4       0
       ..
2235    1
2236    0
2237    0
2238    0
2239    2
Name: Customer_For, Length: 2216, dtype: int32

In [112]:
# checking for value counts in he categorical columns
for col in df.columns:
    if df[col].dtype=="object":
        print(f"{df[col].value_counts()}")



Education
Graduation    1116
PhD            481
Master         365
2n Cycle       200
Basic           54
Name: count, dtype: int64
Marital_Status
Married     857
Together    573
Single      471
Divorced    232
Widow        76
Alone         3
Absurd        2
YOLO          2
Name: count, dtype: int64


In [113]:
# renaming columns in the education column
df["Education"]=df["Education"].replace({"Graduation":"Graduate","PhD":"PostGraduate","Master":"PostGraduate","2n Cycle":"Undergraduate","Basic":"Undergraduate"})
df["Education"].value_counts()

Education
Graduate         1116
PostGraduate      846
Undergraduate     254
Name: count, dtype: int64

In [114]:
# extracting relevant values for the marital status column
df=df[df["Marital_Status"].isin(["Married","Together","Single","Divorced","Widow"])]
df


,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,Customer_For
0,5524,1957,Graduate,Single,58138.0,0,0,2012-09-04,58,635,...,0,0,0,0,0,0,3,11,1,2
1,2174,1954,Graduate,Single,46344.0,1,1,2014-03-08,38,11,...,0,0,0,0,0,0,3,11,0,0
2,4141,1965,Graduate,Together,71613.0,0,0,2013-08-21,26,426,...,0,0,0,0,0,0,3,11,0,1
3,6182,1984,Graduate,Together,26646.0,1,0,2014-02-10,26,11,...,0,0,0,0,0,0,3,11,0,0
4,5324,1981,PostGraduate,Married,58293.0,1,0,2014-01-19,94,173,...,0,0,0,0,0,0,3,11,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,10870,1967,Graduate,Married,61223.0,0,1,2013-06-13,46,709,...,0,0,0,0,0,0,3,11,0,1
2236,4001,1946,PostGraduate,Together,64014.0,2,1,2014-06-10,56,406,...,0,0,0,1,0,0,3,11,0,0
2237,7270,1981,Graduate,Divorced,56981.0,0,0,2014-01-25,91,908,...,0,1,0,0,0,0,3,11,0,0
2238,8235,1956,PostGraduate,Together,69245.0,0,1,2014-01-24,8,428,...,0,0,0,0,0,0,3,11,0,0


In [115]:
# creating a column for the number of children
df["No_children"]=df["Kidhome"]+df["Teenhome"]


In [116]:
# creating a column for total spent by a customer
df["Spent"] = df["MntWines"]+ df["MntFruits"]+ df["MntMeatProducts"]+ df["MntFishProducts"]+ df["MntSweetProducts"]+ df["MntGoldProds"]

In [117]:
# creating an age column
df["Age"]=2024-df["Year_Birth"]
df["Age"]

0       67
1       70
2       59
3       40
4       43
        ..
2235    57
2236    78
2237    43
2238    68
2239    70
Name: Age, Length: 2209, dtype: int64

In [118]:
# renaming columns for clarity 
df=df.rename(columns={"MntWines": "Wines","MntFruits":"Fruits","MntMeatProducts":"Meat","MntFishProducts":"Fish","MntSweetProducts":"Sweets","MntGoldProds":"Gold"})

In [119]:
# removing redundant columns
df.drop(["Dt_Customer", "Z_CostContact", "Z_Revenue", "Year_Birth", "ID"],axis=1,inplace=True)

In [120]:
df.describe()

,Income,Kidhome,Teenhome,Recency,Wines,Fruits,Meat,Fish,Sweets,Gold,...,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,Customer_For,No_children,Spent,Age
count,2209.000000,2209.000000,2209.000000,2209.000000,2209.000000,2209.000000,2209.000000,2209.000000,2209.000000,2209.000000,...,2209.000000,2209.000000,2209.000000,2209.000000,2209.000000,2209.000000,2209.00000,2209.000000,2209.000000,2209.000000
mean,52243.977818,0.441829,0.505206,49.076053,305.194206,26.354912,167.161159,37.556813,27.074694,43.845179,...,0.074242,0.072884,0.063830,0.013581,0.009507,0.149389,0.97148,0.947035,607.186962,55.194658
std,25198.475858,0.537024,0.544318,28.948177,337.688597,39.800145,224.436590,54.583789,41.113282,51.653515,...,0.262223,0.260004,0.244505,0.115769,0.097059,0.356552,0.68472,0.749034,603.354909,11.980704
min,1730.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,5.000000,28.000000
25%,35246.000000,0.000000,0.000000,24.000000,24.000000,2.000000,16.000000,3.000000,1.000000,9.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,0.000000,69.000000,47.000000
50%,51390.000000,0.000000,0.000000,49.000000,174.000000,8.000000,68.000000,12.000000,8.000000,24.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,1.000000,396.000000,54.000000
75%,68627.000000,1.000000,1.000000,74.000000,505.000000,33.000000,233.000000,50.000000,33.000000,56.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,1.000000,1048.000000,65.000000
max,666666.000000,2.000000,2.000000,99.000000,1493.000000,199.000000,1725.000000,259.000000,262.000000,321.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.00000,3.000000,2525.000000,131.000000
